# Using a Python implementation of the Flickr API

This notebook is for the Hack & Yack looking at the BL Flickr account and the Flickr Application Programming Interface (API) - a way of using programming languages to look at and edit the data on Flickr.


The notebook uses the [flickr_api package](https://github.com/alexis-mignon/python-flickr-api/) and the BL Digital Research team's API keys. The notebook has examples looking at the Albums (photosets) available on our account, the tags on those photos, and includes the [DETR-Resnet-50](https://huggingface.co/facebook/detr-resnet-50) object detection model as an example of how we might generate more tags for our images.

In [ ]:
import sys
if "../" not in sys.path:
    sys.path.append("../")
import random
import os

import flickr_api
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import requests
from transformers import DetrImageProcessor, DetrForObjectDetection
import torch

## Flickr API authorisation

*** `key` and `secret` will be shared during the Hack & Yack ***

In [ ]:
key = ""
secret = ""

Follow the link printed below this cell and click "OK, I'll authorize". You will only have read access to the data so there's no risk of making accidental changes! This will take you to a page with some xml on. See below

In [ ]:
flickr_api.set_keys(api_key=key, api_secret=secret)
a = flickr_api.auth.AuthHandler() # creates a new AuthHandler object
perms = "read" # set the required permissions
url = a.get_authorization_url(perms)
print(url) # this is the url we need!

The page you're redirected to will have xml on it like below. Copy the string in the oauth_verifier tag (normally at the bottom of the page).
```xml
<rsp stat="ok">
    <method>flickr.test.echo</method>
    <api_[api_key]acea</api_key>
    <oauth_t[oauth_token]a923d</oauth_token>
    <oauth_verifier>[oauth_verifier]</oauth_verifier>
</rsp>
```

In [ ]:
oauth_verifier = ""  # copy oauth_verifier string in between the speech marks
a.set_verifier(oauth_verifier)
flickr_api.set_auth_handler(a)

## Explore the API

Docs for the [Python implementation](https://github.com/alexis-mignon/python-flickr-api/wiki/API-Reference) of the API and for the [Flickr API](https://www.flickr.com/services/api/) itself.

In [ ]:
# Check the auth has worked by getting info on the BL Flickr account
user = flickr_api.Person.findByEmail("digitalresearch@bl.uk")
user

In [ ]:
photosets = user.getPhotosets()

In [ ]:
# Living with Machines
lwm_ps = photosets[0].getPhotos()
photosets[0].getInfo()

In [ ]:
# Women of the World
wow_ps = photosets[2].getPhotos()
photosets[2].getInfo()

In [ ]:
# Here's one I made earlier
# A dataframe of the results of getInfo on all of our Albums/photosets
info_df = pd.read_csv("../data/processed/photoset_info.csv", index_col="id")

In [ ]:
info_df.head()

## Tags on BL photos

Using the Women of the World album as an example

In [ ]:
wow_tags_df = pd.read_csv("../data/processed/wow_tags.csv")
wow_tags_df

In [ ]:
# sherlock_tags_df = wow_tags_df[wow_tags_df["tag"].str.contains("sherlocknet")].reset_index()

# with open("..\\data\\external\\ofcom_bad_words.txt", "r") as f:
#     bad_words = f.readlines()
#     bad_words = [x.strip("\n").lower() for x in bad_words][1:]

# sherlock_tags_df["tag_val"] = sherlock_tags_df["tag"].apply(lambda x: x.split("=")[1])
# sherlock_tags_df["bad_word"] = sherlock_tags_df["tag_val"].apply(lambda x: x in bad_words)

# sherlock_tags_df[sherlock_tags_df["bad_word"]]["tag_val"].unique()

e.g. BL URL to help find the photo a tag is attached to  
https://www.flickr.com/photos/britishlibrary/11244481994

## DETR-Resnet-50

Using the DETR-Resnet-50 model created by Facebook to try detecting objects in images from your chosen album

This will have to download the model, it will take a few moments

In [ ]:
processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50", revision="no_timm")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50", revision="no_timm")

In [ ]:
def to_rectangle(xy, w, h):
    c = "#{:06x}".format(int(random.random() * 0xffffff))
    p = Rectangle(xy, w, h, ec=c, fill=False)
    return p

def draw_labelled_box(ax, box, label, score):
    xy = (box[0], box[1])
    w, h = box[2] - box[0], box[3] - box[1]
    rct = to_rectangle(xy, w, h)
    ax.add_patch(rct)
    ax.text(box[0] + 0.01*w, box[1] + 0.03*h, s=f"{label} {score}", backgroundcolor="white", va='top')

In [ ]:
url = wow_ps[0].getPhotoFile(size_label="Medium")
image = Image.open(requests.get(url, stream=True).raw)
image

This is the actual processing it might take a minute or two

In [ ]:
inputs = processor(images=image, return_tensors="pt")
outputs = model(**inputs)

In [ ]:
# convert outputs (bounding boxes and class logits) to COCO API
# let's only keep detections with score > 0.9
target_sizes = torch.tensor([image.size[::-1]])
results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.5)[0]

In [ ]:
fig, ax = plt.subplots(figsize=(12,12), subplot_kw={"frameon": False})
ax.imshow(image)
ax.set_axis_off()

for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    box = [round(i, 2) for i in box.tolist()]
    label_text = model.config.id2label[label.item()]
    score_val = round(score.item(), 3)
    print(f"Detected {label_text} with confidence {score_val} at location {box}")
    draw_labelled_box(ax, box, label_text, score_val)